In [264]:
import re
from nltk.stem import PorterStemmer
porter = PorterStemmer()
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
regex = re.compile('[^a-z ]')
regexN = re.compile('[^a-z0-9 ]')


In [265]:
def sanitizeBasic(sent):
    #remove all punctuations and numbers
    sent=regexN.sub(' ', sent)
    sent=re.sub(r"[ ]+", " ", sent)
    return sent

In [266]:
def sanitizeStem(sent):
    #remove all punctuations, numbers and stem as well
    sent=regexN.sub(' ', sent)
    sent=re.sub(r"[ ]+", " ", sent)
    sent= porter.stem(sent)
    return sent

In [267]:
def sanitizeStemStopWords(sent):
    #remove all punctuations, numbers and stop words. Do stemming 
    sent=regexN.sub(' ', sent)
    sent=re.sub(r"[ ]+", " ", sent)
    sent=sent.split(" ")
    sent=' '.join([w for w in sent if w not in stop_words])
    sent= porter.stem(sent)
    
#     sent = re.sub(r"[\*\"“”\n\+\-\/\=\(\)‘•:\[\]\|’\!;\.\,\&\?]+", " ", sent)
#     sent = re.sub(r"[0-9]+", "", sent)
#     sent = re.sub(r"[\']+", "", sent)
#     sent=re.sub(r"[ ]+", " ", sent)
    
    return sent

In [268]:
sanitize=sanitizeStemStopWords

In [269]:
import pandas as pd

In [270]:
data=pd.read_json('Music_reviews_json/reviews_Digital_Music_5.json/Music_Review_train.json', lines=True)
data=data[['reviewText','overall']]
#contains the class data 

In [271]:
num_classes=5

In [272]:
#contains the class data 
df_overall=data[['overall']].astype(int).copy()
# print(df_overall)

In [273]:
#find the frequencies of words in a sentence and returns a dictionary
def findFreq(sent):
    val=sent.split(" ")
    freq={}
    for s in val:
        if s not in freq:
            freq[s]=1
        else:
            freq[s]+=1
    return freq

In [274]:
data['reviewText']=data['reviewText'].str.lower()
# print(dat)

In [275]:
data['reviewText']=data['reviewText'].apply(sanitize)
# print(dat)

In [276]:
import json
lessImpWords=json.load(open('lessImpWords2.json'))
for i in range(1, num_classes+1):
    lessImpWords[i]=set(lessImpWords.pop(str(i)))

In [277]:
def uniqueWords(arr):
    s=set()
    for a in arr:
        if a not in s:
            s.add(a)
    return s

In [278]:
def removeLessImpWords(sent, o):
# def removeLessImpWords(sent ):
    orig=sent.split(' ')
    arr=uniqueWords(orig)
    lessImp=[w for w in arr if w in lessImpWords[o]]
    sent= ' '.join([w for w in orig if w not in lessImp ])
    return sent

In [279]:
print(lessImpWords.keys())

dict_keys([1, 2, 3, 4, 5])


In [280]:
data['reviewText']=data.apply(lambda x: removeLessImpWords(x.reviewText,x.overall) , axis=1)
# data['reviewText']=data.apply(lambda x: removeLessImpWords(x.reviewText, x.overall))

In [281]:
#contains the text data
dat=data[['reviewText']].copy()
# print(dat)

In [282]:
#find the length of each sentence in data
example_length=pd.DataFrame()
example_length['length']=dat['reviewText'].apply(len)
# print(example_length)

In [283]:
# print(dat.iloc[432]['reviewText'])

In [284]:
freq_df=pd.DataFrame()
freq_df['reviewText']=dat['reviewText'].apply(findFreq)
# print(freq_df)

In [285]:
vocab=set()
num_examples=df_overall.shape[0]
for d in range(num_examples):
    for k in freq_df.iloc[d]['reviewText'].keys():
        if k not in vocab:
            vocab.add(k)
# print(len(vocab))

In [286]:
vocab_size=len(vocab)

In [287]:
# print(vocab)

In [288]:
sum_denom={} #the summation of length of reviews depending upon their class

for i in range(1,num_classes+1):
    sum_denom[i]=0
for i in range(num_examples):
    k=df_overall.iloc[i]['overall']
    sum_denom[k]+=example_length.iloc[i]['length']
# print(sum_denom)

In [289]:
sum_numer={} #the sum of the numerator for each theta
for word in vocab:
    sum_numer[word]={}
    for i in range(1,num_classes+1):
        sum_numer[word][i]=0
for i in range(num_examples):
    k=df_overall.iloc[i]['overall']
    d=freq_df.iloc[i]['reviewText']
    for j in d:
        sum_numer[j][k]+=d[j]


In [290]:
import math
thetas={}
for word in vocab:
    thetas[word]={}
    for i in range(1,num_classes+1):
        thetas[word][i]=math.log((sum_numer[word][i]+1)/(sum_denom[i]+vocab_size+1)) # +1 in denominator for unk token(words not in vobabulary)

In [291]:
# print(thetas['have'])
# print(thetas['counterparts'])

In [292]:
#calculate p(y=k)
df_overall_size=df_overall.groupby('overall')
df_overall_size=df_overall_size['overall'].agg(['size'])
df_overall_size['size']=df_overall_size['size']/num_examples
df_overall_size['size']=df_overall_size['size'].apply(math.log)
# print(df['size'])

In [293]:
#unk tokens
unk_p={i:math.log(1/(sum_denom[i]+vocab_size+1)) for i in range(1,num_classes+1)}

In [294]:
#training done. testing
test=pd.read_json('Music_reviews_json/reviews_Digital_Music_5.json/Music_Review_test.json', lines=True)
test['reviewText']=test['reviewText'].copy().str.lower().apply(sanitize)
test['reviewText']=test.apply(lambda x: removeLessImpWords(x.reviewText,x.overall) , axis=1)

In [295]:
testR=pd.DataFrame()
testR['reviewText']=test['reviewText'].str.split(' ')
testO=pd.DataFrame()
testO['overall']=test['overall'].astype(int).copy()

In [296]:
# testR['reviewText']=testR['reviewText'].apply(uniqueWords)

In [297]:
print(testR)

                                              reviewText
0      [hard, believe, memory, trees, came, 11, years...
1      [clasically, styled, introverted, album, memor...
2      [never, thought, enya, would, reach, sublime, ...
3      [third, review, irish, album, write, today, ot...
4      [enya, despite, successful, recording, artist,...
...                                                  ...
13995  [grew, rush, since, 1987, good, album, never, ...
13996  [positive, side, nearly, every, song, cd, leas...
13997  [rush, never, band, stagnate, constantly, expl...
13998  [glassy, relatively, pop, oriented, sound, quo...
13999  [album, elements, old, style, along, elements,...

[14000 rows x 1 columns]


In [298]:
def predictV(setS):
    d={i:df_overall_size['size'][i] for i in range(1, num_classes+1)}
    for s in setS:
        if s not in thetas:
            for i in range(1, num_classes+1):
                d[i]+=unk_p[i]
        else:
            for i in range(1, num_classes+1):
                d[i]+=thetas[s][i]
    return max(d,key=d.get)

In [299]:
# print(dat)

In [300]:
predict=pd.DataFrame()
predict['val']=testR['reviewText'].apply(predictV)

In [301]:
# print(dat)
predictTrain=pd.DataFrame()
predictTrain['val']=(dat['reviewText'].str.split(' ')).apply(predictV)
# predictTrain['val']=dat['reviewText'].apply(uniqueWords).apply(predictV)

In [302]:
# print(predictTrain)

In [303]:
a=0
for i in range(50000):
    if predictTrain.iloc[i]['val']==df_overall.iloc[i]['overall']:
        a+=1;
print("Training Accuracy", a/50000)

Training Accuracy 0.5822


In [304]:
a=0
for i in range(14000):
    if predict.iloc[i]['val']==testO.iloc[i]['overall']:
        a+=1;
print("Testing Accuracy",a/14000)

Testing Accuracy 0.5125714285714286


In [305]:
#confusion matrix
from sklearn import metrics
metrics.confusion_matrix(testO, predict)

array([[ 148,   48,    7,    7,   18],
       [  70,  174,   35,   18,   29],
       [ 110,  305,  337,  156,  178],
       [ 355,  530,  445,  720, 1058],
       [1379, 1004,  379,  693, 5797]])

In [306]:
print(testO.groupby('overall').agg('size'))

overall
1     228
2     326
3    1086
4    3108
5    9252
dtype: int64
